In [ ]:
import os, sys
import miv
import scipy.signal as ss

from miv.io import DataManager
from miv.signal.filter import ButterBandpass, MedianFilter, FilterCollection
from miv.signal.spike import ThresholdCutoff, SpikeSorting, PCADecomposition
from miv.statistics import signal_to_noise, firing_rates
from miv.visualization import plot_frequency_domain, extract_waveforms, plot_waveforms

import elephant.statistics
from viziphant.rasterplot import rasterplot_rates

from tqdm import tqdm

import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
# Experiment name
experiment_query = "experiment0"

# Data call
signal_filter = (
    FilterCollection()
        .append(ButterBandpass(600, 2400, order=4))
        .append(MedianFilter(threshold=60, k=30))
)
spike_detection = ThresholdCutoff(cutoff=5)

# Spike Detection
data_collection = optogenetic.load_data()
data = data_collection.query_path_name(experiment_query)[0]
#false_channels = [12,15,36,41,42,43,45,47,53,57,55,58,61,62]
#data.set_channel_mask(false_channels)
with data.load() as (signal, timestamps, sampling_rate):
    # Preprocess
    signal = signal_filter(signal, sampling_rate)
    spiketrains = spike_detection(signal, timestamps, sampling_rate)

In [ ]:
#Pairwise Granger Causality
import os
import matplotlib.pyplot as plt
import numpy as np
from elephant.causality.granger import pairwise_granger
from viziphant.spike_train_correlation import plot_corrcoef
from miv.typing import SignalType

def pairwise_causality(signal: SignalType, start: float, end: float): 
    
    
    
#Estimates and Plots pairwise Granger Causality

#Parameters
#----------
#signal : SignalType
#    Input signal
#start : float
#    starting point from signal
#end : float
#    End point from signal

#Returns
#-------
#C : Causality Matrix containing directional causalities for X -> Y and Y -> X,
    # instantaneous causality between X,Y, and total causality. X and Y represents electrodes
#figure: plt.Figure
#axes



    p = len(signal[0])
    C = np.zeros((4,p,p))     #Causality Matrix
    q = np.arange(0,p)

    for j in q:
        for k in q:
            if j==k:
                C[:,j,k] = 0 
            else:
                C[:,j,k] = pairwise_granger(np.transpose([signal[start:end,j],signal[start:end,k]]), max_order=1)

    # C or causality matrix contains four p X p matrices. These are directional causalities for X -> Y and Y -> X,
    # instantaneous causality between X,Y, and total causality. X and Y represents electrodes

    #Plotting

    fig, axes = plt.subplots(2,2)
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.6, hspace=0.6)
    plot_corrcoef(C[0], axes=axes[0,0])
    plot_corrcoef(C[1], axes=axes[0,1])
    plot_corrcoef(C[2], axes=axes[1,0])
    plot_corrcoef(C[3], axes=axes[1,1])
    #axes[:,:].set_xlabel('Electrode')
    #axes.set_ylabel('Electrode')
    axes[0, 0].set_ylabel("Electrode")
    axes[1, 0].set_ylabel("Electrode")
    axes[1,0].set_xlabel("Electrode")
    axes[1,1].set_xlabel("Electrode")
    axes[0, 1].set_ylabel("Electrode")
    axes[1, 1].set_ylabel("Electrode")
    axes[0,0].set_xlabel("Electrode")
    axes[0,1].set_xlabel("Electrode")
    axes[0, 0].set_title("Directional causality X => Y")
    axes[0, 1].set_title("Directional causality Y => X")
    axes[1, 0].set_title("Instantaneous causality of X,Y")
    axes[1,1].set_title("Total interdependence of X,Y")
    return C, fig, axes


In [ ]:
pairwise_causality(signal, 0, 1000)

In [1]:
## Welch PSD for an electrode's Signal
## Welch Coherence Estimation between signal X and Y


import os
import matplotlib.pyplot as plt
import numpy as np
from miv.typing import SignalType
from scipy.signal import welch, csd, coherence

def plot_spectral(signal: SignalType, X: float, Y: float, sampling_rate: float, Number_Segments: float):
 


#Plots Power Spectral Densities for channels X and Y, Cross Power Spactral Densities and Coherence between them
#Parameters
#----------
#signal : SignalType
#    Input signal
#X : float
#    Channel 
#Y : float
#    Channel
# sampling_rate : float
#    Sampling frequency
#Number_Segments: float
# Number of segments to divide the entire signal

#Returns
#-------
#figure: plt.Figure
#axes

    L = np.int32(len(signal[:,0])/Number_Segments) # L = length of each segment 
    fs = sampling_rate   # fs = Sampling Frequeny

    fx, Pxx_den = welch(signal[:,X], fs, nperseg=L)  #Welch PSD and frequency for X
    fy, Pyy_den = welch(signal[:,Y], fs, nperseg=L)  #Welch PSD and frequency  for Y
    fxy, Pxy = csd(signal[:,X], signal[:,Y],fs,nperseg=L)   #Welch CSD and frequency for X and Y
    fcxy, Cxy= coherence(signal[:,X], signal[:,Y],fs,nperseg=L) #  Welch Coherence and frequency for X and Y
    #Plotting

    fig, axes = plt.subplots(2,2)
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.6, hspace=0.6)
    axes[0, 0].semilogy(fx, Pxx_den)
    axes[0, 1].semilogy(fy, Pyy_den)
    axes[1, 0].semilogy(fxy, Pxy)
    axes[1, 1].semilogy(fcxy, Cxy)

    #axes[:,:].set_xlabel('Electrode')
    #axes.set_ylabel('Electrode')
    axes[0, 0].set_ylabel("PSD [V**2/Hz]")
    axes[1, 0].set_ylabel("CSD [V**2/Hz]")
    axes[1,0].set_xlabel("'frequency [Hz]'")
    axes[1,1].set_xlabel("'frequency [Hz]'")
    axes[0, 1].set_ylabel("PSD [V**2/Hz]")
    axes[1, 1].set_ylabel("Coherence")
    axes[0,0].set_xlabel("'frequency [Hz]'")
    axes[0,1].set_xlabel("'frequency [Hz]'")
    axes[0, 0].set_title("PSD for X")
    axes[0, 1].set_title("PSD for Y")
    axes[1, 0].set_title("CPSD for X and Y")
    axes[1,1].set_title("Coherence for X,Y")
    return fig, axes